In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64

from dotenv import load_dotenv
import os
import json

# .env 파일을 찾아 환경 변수로 로드
load_dotenv()

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

# ▶️ 반려동물 등록 데이터 가져오기
query = """
SELECT *
FROM companion_animal_registration
"""
df = pd.read_sql(query, engine)

# ▶️ 행정구역 명칭을 folium key와 맞추기 위해 하나로 합치기
df['region'] = df['sigungu']

# 'rank'행 추가
rank_map = {
    '강남구': 1,
    '서초구': 2,
    '송파구': 3,
    '마포구': 4,
    '중구': 5,
    '노원구': 6,
    '서대문구': 7,
    '은평구': 8,
    '강북구': 9,
    '강동구': 10,
    '중랑구': 11
}
df['rank'] = df['sigungu'].map(rank_map)

# 색깔 열 추가
# rank가 있으면 하늘색, 없으면 회색 지정
def color_map(rank):
    if pd.isna(rank):
        return 'yellow'
    else:
        return 'orange'

df['color'] = df['rank'].apply(color_map)
df

,id,sido,sigungu,dog_registered_total,cat_registered_total,total_registered,created_at,region,rank,color
0,1,서울,전체,537884,7278,545162,2025-07-03 04:11:20,전체,NaN,yellow
1,2,서울특별시,강남구,34402,441,34843,2025-07-03 04:11:21,강남구,1.0,orange
2,3,서울특별시,강동구,25292,223,25515,2025-07-03 04:11:21,강동구,10.0,orange
3,4,서울특별시,강북구,18444,141,18585,2025-07-03 04:11:21,강북구,9.0,orange
4,5,서울특별시,강서구,33280,356,33636,2025-07-03 04:11:21,강서구,NaN,yellow
...,...,...,...,...,...,...,...,...,...,...
247,248,경상남도,함양군,1537,3,1540,2025-07-03 04:11:22,함양군,NaN,yellow
248,249,경상남도,합천군,1704,1,1705,2025-07-03 04:11:22,합천군,NaN,yellow
249,250,제주,전체,52807,2965,55772,2025-07-03 04:11:22,전체,NaN,yellow
250,251,제주특별자치도,서귀포시,13367,466,13833,2025-07-03 04:11:22,서귀포시,NaN,yellow


In [2]:
%cd ~/eda-repo-3/

/home/park/eda-repo-3


In [18]:

# ▶️ GeoJSON 경로 지정
geo_path = 'DATA/02. skorea_municipalities_geo_simple.json'  # 반드시 'region' 컬럼과 feature.properties.name이 일치해야 함

# ▶️ GeoJSON 로드 (시도+시군구 이름이 properties.name으로 되어 있어야 함)
with open(geo_path, 'r', encoding='utf-8') as f:
    geo = json.load(f)

# ▶️ 지도 시각화
m = folium.Map(location=[36.5, 127.5], zoom_start=8)




In [9]:
# 스타일 함수 정의
def style_function(feature):
    region_name = feature['properties']['name']
    color = df.loc[df['region'] == region_name, 'color']
    if not color.empty:
        return {'fillColor': color.values[0], 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

# GeoJson 레이어 추가
folium.GeoJson(
    geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'])
).add_to(m)

# # 결과 지도 저장 또는 출력
# m.save('RESULT/visualization/happy_home_folium_choropleth.html')
m

In [22]:
# Folium 지도에 각 지역의 랭크(순위) 표시하기 전체 코드

# import folium
# import json
# import pandas as pd
# import numpy as np
# from shapely.geometry import shape

# 1. GeoJSON 경로 및 데이터 로드
geo_path = 'DATA/02. skorea_municipalities_geo_simple.json'  # 반드시 'region' 컬럼과 feature.properties.name이 일치해야 함
with open(geo_path, 'r', encoding='utf-8') as f:
    geo = json.load(f)

# 2. 랭크 및 색상 데이터 예시 (실제 데이터프레임 df를 준비해야 함)
# 예시: region, rank, color 컬럼이 포함되어 있어야 함
# 아래는 예시 데이터, 실제 데이터로 대체하세요.
# df = pd.DataFrame({
#     'region': ['강남구', '서초구', '송파구', '마포구', '중구', '노원구', '서대문구', '은평구', '강북구', '강동구', '중랑구', '기타구'],
#     'rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, np.nan],
#     'color': ['orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'lightyellow']
# })

# 3. GeoJSON에서 각 구의 중심좌표(centroid) 추출
regions, lats, lngs = [], [], []
for feature in geo['features']:
    region_name = feature['properties']['name']
    geometry = feature['geometry']
    centroid = shape(geometry).centroid
    regions.append(region_name)
    lats.append(centroid.y)
    lngs.append(centroid.x)
geo_df = pd.DataFrame({'region': regions, 'lat': lats, 'lng': lngs})

# 4. 기존 데이터와 중심좌표 병합
df = pd.merge(df, geo_df, on='region', how='left')

# 5. 지도 스타일 함수 정의
def style_function(feature):
    region_name = feature['properties']['name']
    color = df.loc[df['region'] == region_name, 'color']
    if not color.empty:
        return {'fillColor': color.values[0], 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

# 6. 지도 생성 및 GeoJson 레이어 추가
m = folium.Map(location=[37.5665, 126.9780], zoom_start=10)  # 서울 지역 중심
folium.GeoJson(
    geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'])
).add_to(m)

# # 마커 스타일 개선 (더 눈에 잘 보이도록)
# for idx, row in df.iterrows():
#     if not pd.isna(row['rank']):
#         folium.Marker(
#             location=[row['lat'], row['lng']],
#             icon=folium.DivIcon(
#                 html=f"""<div style="
#                     font-size: 16px; 
#                     color: white; 
#                     background: rgba(255,69,0,0.8); 
#                     border-radius: 50%; 
#                     padding: 5px 8px;
#                     font-weight: bold;
#                     text-align: center;
#                     border: 2px solid white;
#                     box-shadow: 0 2px 4px rgba(0,0,0,0.3);
#                 ">{int(row['rank'])}</div>""",
#                 icon_size=(30, 30),
#                 icon_anchor=(15, 15)
#             ),
#             tooltip=f"{row['region']} 순위: {int(row['rank'])}"
#         ).add_to(m)

# 마커 스타일 개선 (더 눈에 잘 보이도록)
for idx, row in df.iterrows():
    if not pd.isna(row['rank']):
        folium.Marker(
            location=[row['lat'], row['lng']],
            icon=folium.DivIcon(
                html=f"""<div style="
                    font-size: 13px; 
                    color: white; 
                    background: rgba(30, 144, 255, 0.8); 
                    border-radius: 15px; 
                    padding: 6px 10px;
                    font-weight: bold;
                    text-align: center;
                    border: 2px solid white;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.3);
                    white-space: nowrap;
                ">{int(row['rank'])}위 {row['region']}</div>""",
                icon_size=(80, 30),
                icon_anchor=(40, 15)
            ),
            tooltip=f"{row['region']} 순위: {int(row['rank'])}"
        ).add_to(m)

# 8. 지도 저장 또는 출력
m.save('RESULT/visualization/happy_home_folium_choropleth_11.html')
m
